In [1]:
# importing libraries

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# reading the data 

def read_data(filename):
    data = pd.read_csv(filename)
    features = data.iloc[:, :-1].values
    labels = data.iloc[:, -1].values - 1 # we'll shift the class labels to 0-25
    return features, labels

# Read the data
data_url = "https://raw.githubusercontent.com/UMBInal/data/main/data.csv"

x, y = read_data(data_url)

In [3]:
# check our data 

x[0]

array([0.01176471, 0.01176471, 0.97647059, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [4]:
y[0]

0

In [5]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [6]:
print(x_train.shape, y_train.shape)

(29335, 10) (29335,)


In [7]:
x_train[0]

array([0.  , 0.1 , 0.  , 0.3 , 0.  , 0.05, 0.1 , 0.15, 0.05, 0.25])

In [8]:
# Reshape the data
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

In [9]:
x_train[0]

array([[0.  ],
       [0.1 ],
       [0.  ],
       [0.3 ],
       [0.  ],
       [0.05],
       [0.1 ],
       [0.15],
       [0.05],
       [0.25]])

In [10]:
# Defining input shape and number of classes
input_shape = (x_train.shape[1], 1)
num_classes = len(np.unique(y))

In [11]:
print(input_shape, num_classes)

(10, 1) 26


In [ ]:
# Transformer Encoder function
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs
    
    x = layers.Dense(ff_dim, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Dense(inputs.shape[-1])(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res

In [ ]:
# Transformer Decoder function
def transformer_decoder(inputs, enc_output, head_size, num_heads, ff_dim, dropout=0):
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)

    # Attend to the encoder output
    x_attended = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, enc_output)
    x_attended = layers.Dropout(dropout)(x_attended)
    x_attended = layers.LayerNormalization(epsilon=1e-6)(x_attended)


    res = x_attended + x
    x_ff = layers.Dense(ff_dim, activation="relu")(res)
    x_ff = layers.Dropout(dropout)(x_ff)
    x_ff = layers.Dense(inputs.shape[-1])(x_ff)
    x = layers.LayerNormalization(epsilon=1e-6)(x_ff)

    return x + res

In [ ]:
# Build the model
def build_model(
    input_shape, 
    head_size, 
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    
# Encoder
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

        enc_output = x
    
# Decoder
    enc_output = x  # Output from the Encoder
    for _ in range(num_transformer_blocks):
        x = transformer_decoder(x, enc_output, head_size, num_heads, ff_dim, dropout)

# Final Steps
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)

    outputs = layers.Dense(num_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)


In [ ]:
# Define the hyperparameters
head_size = 2
num_heads = 8
ff_dim = 64
num_transformer_blocks = 6
mlp_units = [64]
dropout = 0.25
mlp_dropout = 0.4

In [ ]:
# declare the model
model = build_model(input_shape,
                    head_size, num_heads,
                    ff_dim,
                    num_transformer_blocks,
                    mlp_units,
                    dropout,
                    mlp_dropout)


In [ ]:
# Compile the model
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
# Train the model
model.fit(x_train, y_train, batch_size=64, epochs=100, validation_split=0.2)

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")